# 🚀 Introdução ao Streamlit — Do Zero

## 1. O que é Python?
- É uma **linguagem de programação** simples e muito usada em **Ciência de Dados** e **Inteligência Artificial**.
- Você escreve comandos (código) e o Python executa.
- Exemplo simples:
  - `print("Olá, mundo!")` → imprime a frase "Olá, mundo!" na tela.

## 2. O que é o Streamlit?
- O **Streamlit** é uma **biblioteca** (pacote pronto) do Python para criar **aplicações web interativas** de forma **muito fácil**.
- Você escreve código Python e o Streamlit transforma isso em uma página web.
- É muito usado para **painéis interativos**, **dashboards** e **projetos de IA**.

## 3. Como instalar o Streamlit
No terminal ou no Jupyter Notebook (em uma célula):
- `pip install streamlit`

## 4. Como criar um app básico com Streamlit
1. **Crie um arquivo** chamado `meu_app.py`.
2. **Escreva o código**:
   - `import streamlit as st`
   - `st.title("Meu primeiro app Streamlit 🚀")`
   - `st.write("Este é um app interativo feito em Python.")`
   - `nome = st.text_input("Digite seu nome:")`
   - `if nome:`
       - `st.success(f"Olá, {nome}!")`
3. **Rodar o app**:
   - `streamlit run meu_app.py`
4. O navegador vai abrir e mostrar seu app.

## 5. Conceitos básicos do Streamlit
- `st.title("texto")` → Cria um título.
- `st.write("texto ou variável")` → Mostra texto ou dados.
- `st.text_input("pergunta")` → Caixa de texto para o usuário digitar.
- `st.button("Botão")` → Cria um botão.
- `st.file_uploader("Mensagem")` → Caixa para enviar arquivos.
- `st.dataframe(df)` → Mostra uma tabela de dados.

## 6. Nosso exemplo do Titanic (passo-a-passo)
1. Vamos usar o **dataset Titanic** (arquivo `.csv` do Kaggle).
2. O usuário vai **enviar o CSV** pelo app.
3. O app vai:
   - Ler os dados com o **pandas**.
   - Treinar um **modelo de IA** simples (Regressão Logística ou Linear).
   - Mostrar as **variáveis mais importantes** para prever:
     - Se uma pessoa sobreviveu (`Survived`).
     - Ou o preço da passagem (`Fare`).
4. O app responde de forma **interativa** no navegador.

## 7. Como funciona o fluxo do app
1. **Usuário envia dados** → `st.file_uploader`
2. **Código lê e prepara** → pandas, limpeza, codificação de categorias.
3. **Modelo de IA é treinado** → scikit-learn.
4. **Resultados e métricas aparecem** → `st.dataframe`, `st.write`.
5. **Chatbot dá a resposta final** → função que interpreta e mostra.

## 8. Vantagens do Streamlit
- Fácil de aprender (poucas linhas de código).
- Rápido para criar protótipos.
- Não precisa saber HTML/CSS/JavaScript.
- Funciona com qualquer biblioteca Python.

## 9. Próximos passos para o aluno
- Entender os comandos básicos do Python.
- Aprender a usar bibliotecas (como pandas e scikit-learn).
- Criar pequenos apps no Streamlit.
- Conectar esses apps a modelos de IA e bases de dados.

💡 **Resumo:**  
O Streamlit é como uma "ponte" entre seu código Python e uma página web bonita e interativa.  
Com ele, podemos mostrar para qualquer pessoa como um modelo de IA funciona — sem precisar instalar nada complicado no computador dela.


In [3]:
# ============================================================
# CÉLULA 1 — Instalar dependências (rode uma vez)
# ============================================================
# Obs.: se seu Jupyter usa um venv/conda, não use sudo aqui.
import sys, subprocess

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--break-system-packages", *pkgs])

pip_install(["streamlit", "pandas", "numpy", "scikit-learn"])


Defaulting to user installation because normal site-packages is not writeable


In [4]:
# ============================================================
# CÉLULA 2 — Definir o código do app Streamlit (Titanic Chatbot) — robusto
# ============================================================
app_code = r'''
# streamlit_app.py — Chatbot Titanic (v1) para Jupyter — robusto
import streamlit as st
import pandas as pd
import numpy as np
from typing import Tuple, List

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error
)

st.set_page_config(page_title="Chatbot Titanic IA — Jupyter", page_icon="🚢", layout="wide")
st.title("🚢 Chatbot Titanic — Fundamentos de IA (Jupyter + Streamlit)")

with st.sidebar:
    st.header("⚙️ Configuração")
    up = st.file_uploader("Faça upload do CSV (ex.: train.csv do Titanic)", type=["csv"])
    alvo = st.selectbox("Alvo (target)", ["Survived", "Fare"], index=0,
                        help="Survived → classificação (logística); Fare → regressão (linear)")
    pergunta = st.text_input("Pergunta (ex.: quais características de quem sobrevive?)",
                             value="Quais características de quem sobrevive?")
    test_size = st.slider("Proporção de teste", 0.1, 0.4, 0.2, 0.05)
    random_state = st.number_input("random_state", value=42, step=1)
    btn = st.button("🚀 Rodar")

st.markdown("""
**O que este app faz?**
1. Você envia a base (ex.: `train.csv` do Kaggle Titanic).
2. Escolhe o alvo:
   - `Survived` → classificação (Regressão Logística).
   - `Fare` → regressão (Regressão Linear).
3. Faz sua pergunta.
4. O app treina o modelo (imputação, one-hot, escala) e responde com **características mais relevantes** e **métricas**.
""")

# ---------- Helpers de compatibilidade e pré-processamento ----------
def make_ohe():
    """Cria OneHotEncoder compatível com sklearn novo e antigo."""
    try:
        # sklearn >= 1.2 (1.4 removeu 'sparse'; usa 'sparse_output')
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        # sklearn <= 1.1
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

def build_preprocessor(X: pd.DataFrame) -> Tuple[ColumnTransformer, List[str], List[str]]:
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

    numeric_processor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ])

    categorical_processor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", make_ohe()),  # compatível com qualquer versão
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_processor, numeric_cols),
            ("cat", categorical_processor, categorical_cols),
        ],
        remainder="drop"
    )
    return preprocessor, numeric_cols, categorical_cols

# ---------- Importância/explicação ----------
def feature_importance_from_logistic(model: Pipeline) -> pd.DataFrame:
    preproc = model.named_steps["preprocessor"]
    clf = model.named_steps["classifier"]

    # nomes pós-transformação
    try:
        feat_names = preproc.get_feature_names_out()
    except Exception:
        try:
            num_cols = preproc.transformers_[0][2]
        except Exception:
            num_cols = []
        try:
            cat_cols = preproc.transformers_[1][2]
            ohe = preproc.named_transformers_["cat"].named_steps["onehot"]
            cat_feat = ohe.get_feature_names_out(cat_cols)
        except Exception:
            cat_feat = []
        feat_names = np.array(list(num_cols) + list(cat_feat))

    coefs = clf.coef_[0] if hasattr(clf, "coef_") else np.full(len(feat_names), np.nan)
    imp = pd.DataFrame({"Feature": feat_names, "Coeficiente": coefs})
    imp["Odds_Ratio"] = np.exp(imp["Coeficiente"])
    imp["Abs"] = np.abs(imp["Coeficiente"])
    return imp.sort_values("Abs", ascending=False)

def regression_importance_from_linear(model: Pipeline, X_train: pd.DataFrame) -> pd.DataFrame:
    preproc = model.named_steps["preprocessor"]
    reg = model.named_steps["regressor"]

    # nomes pós-transformação
    try:
        feat_names = preproc.get_feature_names_out()
    except Exception:
        try:
            num_cols = preproc.transformers_[0][2]
        except Exception:
            num_cols = []
        try:
            cat_cols = preproc.transformers_[1][2]
            ohe = preproc.named_transformers_["cat"].named_steps["onehot"]
            cat_feat = ohe.get_feature_names_out(cat_cols)
        except Exception:
            cat_feat = []
        feat_names = np.array(list(num_cols) + list(cat_feat))

    # NÃO refaça fit: o preproc já está ajustado dentro do pipeline após model.fit
    Xtr_trans = preproc.transform(X_train)
    std_x = Xtr_trans.std(axis=0)

    coefs = reg.coef_
    impacto = np.abs(coefs) * std_x

    imp = pd.DataFrame({"Feature": feat_names, "Coef_Linear": coefs, "Impacto_Relativo": impacto})
    imp["AbsCoef"] = np.abs(coefs)
    return imp.sort_values("Impacto_Relativo", ascending=False)

def answer_from_question(pergunta: str, tabela_imp: pd.DataFrame, tarefa: str) -> str:
    p = (pergunta or "").lower()
    if any(k in p for k in ["característica","caracteristicas","feature","variável","variaveis","importante","importância","importancia"]):
        if tarefa == "classificacao":
            top_pos = tabela_imp.sort_values("Coeficiente", ascending=False).head(3)[["Feature","Coeficiente","Odds_Ratio"]]
            top_neg = tabela_imp.sort_values("Coeficiente", ascending=True).head(3)[["Feature","Coeficiente","Odds_Ratio"]]
            return (
                "🔎 **Características mais associadas à sobrevivência**\n\n"
                f"⬆️ Aumentam a chance (top 3):\n{top_pos.to_markdown(index=False)}\n\n"
                f"⬇️ Diminuem a chance (top 3):\n{top_neg.to_markdown(index=False)}"
            )
        else:
            top_imp = tabela_imp[["Feature","Coef_Linear","Impacto_Relativo"]].head(5)
            return (
                "🔎 **Características mais associadas ao alvo (regressão)**\n\n"
                f"Top 5 por impacto relativo:\n{top_imp.to_markdown(index=False)}"
            )
    return "Posso analisar **importância das variáveis** e **métricas**. Tente: *quais características são mais importantes?*"

# ========== Fluxo da UI ==========
if not up and not btn:
    st.info("⬅️ Envie um CSV (ex.: `train.csv`) e/ou clique em **Rodar** após configurar.")
elif btn and not up:
    st.warning("⚠️ Você clicou em **Rodar**, mas nenhum CSV foi enviado. Anexe um arquivo primeiro.")
elif up:
    # Leitura robusta do CSV enviado
    try:
        # sep=None (engine='python') tenta inferir separador automaticamente
        df = pd.read_csv(up, sep=None, engine="python")
        st.success("✅ Arquivo carregado com sucesso!")
    except Exception as e:
        st.error(f"Erro ao ler CSV: {e}")
        st.stop()

    st.subheader("👀 Pré-visualização dos dados")
    st.dataframe(df.head(10), use_container_width=True)

    cols_sug = ["Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
    disp_cols = [c for c in cols_sug if c in df.columns]
    if len(disp_cols) >= 3:
        st.caption("Sugestão de colunas (Titanic padrão): " + ", ".join(disp_cols))

    if btn:
        if alvo not in df.columns:
            st.error(f"O alvo '{alvo}' não existe no CSV.")
            st.stop()

        # X = todas menos alvo e ids comuns
        drop_candidates = [alvo, "PassengerId", "Name", "Ticket", "Cabin"]
        X_full = df.drop(columns=[c for c in drop_candidates if c in df.columns], errors="ignore")
        y_full = df[alvo]

        # Garantir valores válidos no alvo
        mask_notna = y_full.notna()
        X_full, y_full = X_full[mask_notna], y_full[mask_notna]

        # Definição do pipeline conforme tarefa
        preproc, _, _ = build_preprocessor(X_full)
        if alvo == "Survived":
            if y_full.nunique() < 2:
                st.error("O alvo 'Survived' precisa ter ao menos duas classes (0 e 1) após limpeza.")
                st.stop()
            tarefa = "classificacao"
            model = Pipeline(steps=[
                ("preprocessor", preproc),
                ("classifier", LogisticRegression(max_iter=1000))
            ])
            strat = y_full
        else:
            tarefa = "regressao"
            model = Pipeline(steps=[
                ("preprocessor", preproc),
                ("regressor", LinearRegression())
            ])
            strat = None

        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X_full, y_full, test_size=float(test_size), random_state=int(random_state),
                stratify=strat if tarefa=="classificacao" else None
            )
        except ValueError as e:
            st.error(f"Erro ao separar treino/teste: {e}")
            st.stop()

        # Treinar
        model.fit(X_train, y_train)

        # Saídas
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("### 📈 Importância / Coeficientes")
            if tarefa == "classificacao":
                imp = feature_importance_from_logistic(model)
                st.dataframe(imp[["Feature","Coeficiente","Odds_Ratio","Abs"]].head(20), use_container_width=True)
            else:
                imp = regression_importance_from_linear(model, X_train)
                st.dataframe(imp[["Feature","Coef_Linear","Impacto_Relativo","AbsCoef"]].head(20), use_container_width=True)

        with col2:
            st.markdown("### 📊 Métricas")
            if tarefa == "classificacao":
                y_pred = model.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                prec = precision_score(y_test, y_pred, zero_division=0)
                rec = recall_score(y_test, y_pred, zero_division=0)
                f1 = f1_score(y_test, y_pred, zero_division=0)
                cm = confusion_matrix(y_test, y_pred)
                mets = pd.DataFrame({"Acurácia":[acc], "Precisão":[prec], "Recall":[rec], "F1":[f1]})
                st.dataframe(mets, use_container_width=True)
                st.write("Matriz de Confusão:")
                st.dataframe(pd.DataFrame(cm, index=["y=0","y=1"], columns=["ŷ=0","ŷ=1"]))
            else:
                y_pred = model.predict(X_test)
                rmse = mean_squared_error(y_test, y_pred, squared=False)
                st.write(f"RMSE (teste): **{rmse:.4f}**")

        st.markdown("### 💬 Resposta do Chatbot")
        st.info(answer_from_question(pergunta, imp, tarefa))
        st.success("Pronto! Você pode ajustar a pergunta, trocar o alvo e reenviar a base.")
'''


In [5]:
import os, sys, time, subprocess, socket
from IPython.display import IFrame, display
from pathlib import Path

app_path = Path("streamlit_app.py").resolve()

def get_free_port(start=8501, end=8599):
    for port in range(start, end+1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("127.0.0.1", port)) != 0:
                return port
    raise RuntimeError("Sem portas livres entre 8501-8599.")

PORT = get_free_port()
url = f"http://127.0.0.1:{PORT}"

cmd = [
    sys.executable, "-m", "streamlit", "run", str(app_path),
    "--server.headless", "true",
    "--server.port", str(PORT),
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
    "--browser.gatherUsageStats", "false",
    "--server.fileWatcherType", "none",
]

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print(f"Iniciando Streamlit em {url} ...")
time.sleep(4.0)  # pequeno delay para subir
display(IFrame(src=url, width="100%", height=720))


Iniciando Streamlit em http://127.0.0.1:8501 ...


In [6]:
# ============================================================
# CÉLULA 5 — Encerrar o servidor (rode quando quiser parar)
# ============================================================
try:
    proc.terminate()
except Exception:
    pass
